#1. Install Dependencies
First install the libraries needed to execute recipes, this only needs to be done once, then click play.


In [ ]:
!pip install git+https://github.com/google/starthinker


#2. Get Cloud Project ID
To run this recipe [requires a Google Cloud Project](https://github.com/google/starthinker/blob/master/tutorials/cloud_project.md), this only needs to be done once, then click play.


In [ ]:
CLOUD_PROJECT = 'PASTE PROJECT ID HERE'

print("Cloud Project Set To: %s" % CLOUD_PROJECT)


#3. Get Client Credentials
To read and write to various endpoints requires [downloading client credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_client_installed.md), this only needs to be done once, then click play.


In [ ]:
CLIENT_CREDENTIALS = 'PASTE CREDENTIALS HERE'

print("Client Credentials Set To: %s" % CLIENT_CREDENTIALS)


#4. Enter DV360 Bulk Targeting Editor Parameters
Allows bulk targeting DV360 through Sheets and BigQuery.
 1. Select <b>Load</b>, click <b>Save + Run</b>, a sheet called <b> DV Targeter</b> will be created.
 1. In the <b>Partners</b> sheet tab, fill in <i>Filter</i> column then select <b>Load</b>, click <b>Save + Run</b>.
 1. In the <b>Advertisers</b> sheet tab, fill in <i>Filter</i> column. then select <b>Load</b>, click <b>Save + Run</b>.
 1. Check the First And Third Party option to load audiences, which may be slow.  If not loaded, user will enter audience ids into the sheet manually.
 1. On the <b>Line Items</b> sheet tab, the <i>Filter</i> is used only to limit drop down choices in the rest of the tool.
 1. Optionally edit or filter the <i>Targeting Options</i> or <i>Inventory Sources</i> sheets to limit choices.
 1. Make targeting updates, fill in changes on all tabs with colored fields (RED FIELDS ARE NOT IMPLEMENTED, IGNORE).
 1. Select <i>Preview</i>, click <b>Save + Run</b> then check the <b>Preview</b> tabs.
 1. Select <b>Update</b>, click <b>Save + Run</b> then check the <b>Success</b> and <b>Error</b> tabs.
 1. Load and Update can be run multiple times.
 1. If an update fails, all parts of the update failed, break it up into multiple updates.
 1. To refresh the Partner, Advertiser, or Line Item list, remove the filters and run load.
Modify the values below for your use case, can be done multiple times, then click play.


In [ ]:
FIELDS = {
  'auth_dv': 'user',  # Credentials used for dv.
  'auth_sheet': 'user',  # Credentials used for sheet.
  'auth_bigquery': 'service',  # Credentials used for bigquery.
  'recipe_name': '',  # Name of Google Sheet to create.
  'recipe_slug': '',  # Name of Google BigQuery dataset to create.
  'command': 'Load',  # Action to take.
  'first_and_third': False,  # Load first and third party data (may be slow). If not selected, enter audience identifiers into sheet manually.
}

print("Parameters Set To: %s" % FIELDS)


#5. Execute DV360 Bulk Targeting Editor
This does NOT need to be modified unless you are changing the recipe, click play.


In [ ]:
from starthinker.util.configuration import Configuration
from starthinker.util.configuration import commandline_parser
from starthinker.util.configuration import execute
from starthinker.util.recipe import json_set_fields

USER_CREDENTIALS = '/content/user.json'

TASKS = [
  {
    'dataset': {
      '__comment__': 'Ensure dataset exists.',
      'auth': 'user',
      'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 2,'default': '','description': 'Name of Google BigQuery dataset to create.'}}
    }
  },
  {
    'drive': {
      '__comment__': 'Copy the default template to sheet with the recipe name',
      'auth': 'user',
      'copy': {
        'source': 'https://docs.google.com/spreadsheets/d/1ARkIvh0D-gltZeiwniUonMNrm0Mi1s2meZ9FUjutXOE/',
        'destination': {'field': {'name': 'recipe_name','suffix': ' DV Targeter','kind': 'string','order': 3,'default': '','description': 'Name of Google Sheet to create.'}}
      }
    }
  },
  {
    'dv_targeter': {
      '__comment': 'Depending on users choice, execute a different part of the solution.',
      'auth_dv': {'field': {'name': 'auth_dv','kind': 'authentication','order': 1,'default': 'user','description': 'Credentials used for dv.'}},
      'auth_sheets': {'field': {'name': 'auth_sheet','kind': 'authentication','order': 2,'default': 'user','description': 'Credentials used for sheet.'}},
      'auth_bigquery': {'field': {'name': 'auth_bigquery','kind': 'authentication','order': 3,'default': 'service','description': 'Credentials used for bigquery.'}},
      'sheet': {'field': {'name': 'recipe_name','suffix': ' DV Targeter','kind': 'string','order': 4,'default': '','description': 'Name of Google Sheet to create.'}},
      'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 5,'default': '','description': 'Name of Google BigQuery dataset to create.'}},
      'command': {'field': {'name': 'command','kind': 'choice','choices': ['Clear','Load','Preview','Update'],'order': 6,'default': 'Load','description': 'Action to take.'}},
      'first_and_third': {'field': {'name': 'first_and_third','kind': 'boolean','order': 6,'default': False,'description': 'Load first and third party data (may be slow). If not selected, enter audience identifiers into sheet manually.'}}
    }
  }
]

json_set_fields(TASKS, FIELDS)

execute(Configuration(project=CLOUD_PROJECT, client=CLIENT_CREDENTIALS, user=USER_CREDENTIALS, verbose=True), TASKS, force=True)
